# Blob Detector

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import cv2
import glob
import copy
import numpy as np

from typing import Tuple, List
from itertools import chain

import matplotlib.pyplot as plt

import matplotlib as mpl

mpl.rc("figure", dpi=200)

In [ ]:
SAMPLE_IMG: int = 7

this_dir: str = os.path.abspath('')
assets_dir: str = os.path.join(this_dir, "..", "..", "assets", "notebooks", "01.computer-vision", "blob-detector", "train")
image_glob_fpath: str = os.path.join(assets_dir, "*.jpg")
image0_fpath: str = os.path.join(assets_dir, f"{SAMPLE_IMG}.jpg")

all_images: List[str] = sorted(glob.glob(image_glob_fpath), key=lambda s: int(os.path.basename(s).split('.')[0]))

## Sample image

In [ ]:
sample: np.ndarray = cv2.imread(image0_fpath)
plt.figure(figsize=(16, 9), dpi=96)
rgb = cv2.cvtColor(sample, cv2.COLOR_BGR2RGB)
plt.imshow(rgb)

## Blob Detector

We are going to use the class `SimpleBlobDetector` from OpenCV to detect the blobs of light that the lit LEDs generate in the image. 

For further details about how this detector works internally, we suggest reading the [official documentation](https://docs.opencv.org/3.4/d0/d7a/classcv_1_1SimpleBlobDetector.html#details).

In [ ]:
default_parameters: dict = {
    "minThreshold": 5,
    "maxThreshold": 75,
    "thresholdStep": 10,

    # Filter by Area.
    "filterByArea": True,
    "minArea": (8 ** 2) * 3.14,    # min 8 pixels diameter
    "maxArea": (64 ** 2) * 3.14,   # max 64 pixels diameter

    # Filter by Circularity
    "filterByCircularity": True,
    "minCircularity": 0.7,

    # Filter by Convexity
    "filterByConvexity": True,
    "minConvexity": 0.8,

    # Filter by Inertia
    "filterByInertia": False,
    "minInertiaRatio": 0.05,
}


def extract_hsv(_bgr: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    _hsv = cv2.cvtColor(_bgr, cv2.COLOR_BGR2HSV)
    _h = _hsv[:,:,0]
    _s = _hsv[:,:,1]
    _v = _hsv[:,:,2]
    return _h, _s, _v


def detect_blobs(_bgr: np.ndarray, **_params) -> Tuple[cv2.KeyPoint, ...]:
    # create new configuration that extends the default
    _cfg = cv2.SimpleBlobDetector_Params()
    for _k, _v in chain(default_parameters.items(), _params.items()):
        setattr(_cfg, _k, _v)
    # create detector
    _detector = cv2.SimpleBlobDetector.create(_cfg)
    # find blobs
    _keypoints = _detector.detect(_bgr)
    print(f"Found {len(_keypoints)} blobs")
    return _keypoints


def draw_blobs(_bgr: np.ndarray, _blobs: Tuple[cv2.KeyPoint, ...]) -> np.ndarray:
    # draw blobs on the original image
    _bgr1 = copy.deepcopy(_bgr)
    for kp in _blobs:
        _bgr1 = cv2.circle(_bgr1, tuple(map(int, kp.pt)), int(kp.size), (0, 255, 0), 3)
    return _bgr1


def show_image(_bgr: np.ndarray, _title: str = "ND"):
    # show image with superimposed blobs
    plt.figure(figsize=(16, 9), dpi=96)
    _rgb = cv2.cvtColor(_bgr, cv2.COLOR_BGR2RGB)
    plt.imshow(_rgb)
    plt.title(f'Image: {_title}')


def test_all(**_params):
    _rows, _cols = 4, 3
    _f, _axs = plt.subplots(_rows, _cols, figsize=(20, 25))
    _f.set_tight_layout(True)
    _total: int = 0

    for _i, _img_fpath in enumerate(all_images):
        _ax = _axs[_i // _cols][_i - (_i // _cols) * _rows]
        _fname: str = os.path.basename(_img_fpath)
        print("Image: ", _fname, end="; ")
        _bgr: np.ndarray = cv2.imread(_img_fpath)
        _, _s, _ = extract_hsv(_bgr)
        _blobs: Tuple[cv2.KeyPoint, ...] = detect_blobs(_s, **_params)
        _debug_bgr: np.ndarray = draw_blobs(_bgr, _blobs)
        _ax.set_title(f"Image: {_fname}")
        _rgb = cv2.cvtColor(_debug_bgr, cv2.COLOR_BGR2RGB)
        _ax.imshow(_rgb)
        _total += len(_blobs)
    
    print(f"----\nFound {_total} blobs.")
    

## Test detector on sample image (using full BGR image)

In [ ]:
blobs: Tuple[cv2.KeyPoint, ...] = detect_blobs(sample)
debug_img: np.ndarray = draw_blobs(sample, blobs)
show_image(debug_img, "detection on BGR")

## Intuition: Lit LEDs appear to be saturating the BGR channels of the image

Let us take a look at the HSV channels instead of BGR.

In [ ]:
h, s, v = extract_hsv(sample)
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (20,10))

ax1.set_title("Hue")
ax1.imshow(h, cmap = "gray")
ax2.set_title("Saturation")
ax2.imshow(s, cmap = "gray")
ax3.set_title("Value")
ax3.imshow(v, cmap = "gray")

**NOTE:** The Saturation channel appears to have clear peaks (black circles) in the places corresponding to the lit LEDs. Let us try the blob detector on that channel only.

## Test detector on sample image (using Saturation channel from HSV)

In [ ]:
_, s, _ = extract_hsv(sample)
blobs: Tuple[cv2.KeyPoint, ...] = detect_blobs(s)
debug_img: np.ndarray = draw_blobs(sample, blobs)
show_image(debug_img, "detection on Saturation channel")

## Experiment with the parameters

You can pass to the function `detect_blobs()` any parameter from the dictionary `default_parameters` above to replace the default value.

In [ ]:
_, s, _ = extract_hsv(sample)
blobs: Tuple[cv2.KeyPoint, ...] = detect_blobs(
    s,
    # # ---- generic
    minThreshold=10,
    # maxThreshold=75,
    # thresholdStep=10,
    # # ---- filter by area
    # filterByArea=True,
    # minArea=(8 ** 2) * 3.14,
    # maxArea=(64 ** 2) * 3.14,
    # # ---- filter by circularity
    # filterByCircularity=True,
    # minCircularity=0.7,
    # # ---- filter by convexity
    # filterByConvexity=True,
    # minConvexity=0.8,
    # # ---- filter by inertia
    # filterByInertia=False,
    # minInertiaRatio=0.05,
)
debug_img: np.ndarray = draw_blobs(sample, blobs)
show_image(debug_img, "detection on Saturation channel; minThreshold = 10")

In [ ]:
_, s, _ = extract_hsv(sample)
blobs: Tuple[cv2.KeyPoint, ...] = detect_blobs(
    s,
    # # ---- generic
    minThreshold=10,
    # maxThreshold=75,
    # thresholdStep=10,
    # # ---- filter by area
    # filterByArea=True,
    # minArea=(8 ** 2) * 3.14,
    # maxArea=(64 ** 2) * 3.14,
    # # ---- filter by circularity
    # filterByCircularity=True,
    minCircularity=0.6,
    # # ---- filter by convexity
    # filterByConvexity=True,
    # minConvexity=0.8,
    # # ---- filter by inertia
    # filterByInertia=False,
    # minInertiaRatio=0.05,
)
debug_img: np.ndarray = draw_blobs(sample, blobs)
show_image(debug_img, "detection on Saturation channel; minThreshold=10, minCircularity=0.6")

## Test on the entire train set (with default parameters)

In [ ]:
test_all()

## Test on the entire train set (with better parameters)

In [ ]:
test_all(
    # # ---- generic
    minThreshold=10,
    # maxThreshold=75,
    # thresholdStep=10,
    # # ---- filter by area
    # filterByArea=True,
    # minArea=(8 ** 2) * 3.14,
    # maxArea=(64 ** 2) * 3.14,
    # # ---- filter by circularity
    # filterByCircularity=True,
    minCircularity=0.6,
    # # ---- filter by convexity
    # filterByConvexity=True,
    # minConvexity=0.8,
    # # ---- filter by inertia
    # filterByInertia=False,
    # minInertiaRatio=0.05,
)